# Complementaria Repaso Quiz 3 - Jueves 12:30

In [1]:
import numpy as np
import pandas as pd
from jmarkov.mdp.dtmdp import dtmdp

In [2]:
# Matrices de transición (orden de estados: B, M, A)

P_P = np.array([
    [0.80, 0.18, 0.02],
    [0.30, 0.55, 0.15],
    [0.10, 0.50, 0.40]
])

P_G = np.array([
    [0.55, 0.35, 0.10],
    [0.15, 0.55, 0.30],
    [0.05, 0.30, 0.65]
])

P_L = np.array([
    [0.70, 0.25, 0.05],
    [0.25, 0.55, 0.20],
    [0.10, 0.45, 0.45]
])

P_S = np.array([
    [0.90, 0.10, 0.00],
    [0.50, 0.45, 0.05],
    [0.25, 0.55, 0.20]
])

# Retornos inmediatos R(s,a) en miles de dólares
retornos = np.array([#P   #G  #L  #S
                    [60, 110, 70, 10],   # B
                    [40,  80, 60,  5],   # M
                    [20,  40, 35,  0]    # A
                    ], dtype=float)


## Parte A: Implementación del MDP

### 1. Espacio de estados, acciones, matrices de transición, matriz de retornos y objeto MDP

In [ ]:
# Espacio de estados
s_x = np.array(["Baja", "Media", "Alta"])

# Espacio de acciones
a_i = np.array(["Operar en pequeño", "Operar en grande", "Diversificar / Lavar", "Pausar operaciones"])
    
matrices = {}
matrices["Operar en pequeño"] = P_P
matrices["Operar en grande"] = P_G
matrices["Diversificar / Lavar"] = P_L
matrices["Pausar operaciones"] = P_S

# Matriz de retornos
retornos = np.array([  # P    G    L    S
                    [60, 110, 70, 10],   # B
                    [40,  80, 60,  5],   # M
                    [20,  40, 35,  0]    # A
                    ], dtype=float)

# Crear el objeto MDP
mdp = dtmdp(s_x, a_i, matrices, retornos, discount_factor=0.95)

value_functions: [1532.63862521 1467.05680641 1410.94649533]
optimal_policy: {np.str_('Baja'): np.str_('Operar en grande'), np.str_('Media'): np.str_('Operar en grande'), np.str_('Alta'): np.str_('Diversificar / Lavar')}


### 2. Valores óptimos para cada estado usando iteración de política

In [24]:
value_functions, optimal_policy = mdp.solve(tolerance=0, minimize=False, method="policy_iteration")

print(f"value_functions: {value_functions}")

value_functions: [1532.65965066 1467.07725411 1410.96627588]


### 3. Obtenga la política óptima

In [25]:
print(f"optimal_policy: {optimal_policy}")

optimal_policy: {np.str_('Baja'): np.str_('Operar en grande'), np.str_('Media'): np.str_('Operar en grande'), np.str_('Alta'): np.str_('Diversificar / Lavar')}


### 4. Calcule el valor esperado de la política óptima en el largo plazo

In [28]:
mdp.expected_policy_value(value_functions, optimal_policy)


np.float64(1464.999999864065)

### 5. Genere la matriz de transición inducida por la política óptima

In [40]:
mdp.policy_transition_matrix(optimal_policy)

pd.DataFrame(mdp.policy_transition_matrix(optimal_policy))

,0,1,2
0,0.55,0.35,0.10
1,0.15,0.55,0.30
2,0.10,0.45,0.45


## Parte B: Simulación de Monte Carlo
Realice una smulación de Monte Carlo con 1000 escenarios, cada uno con 25 semanas, inciiando en el estado Media (M) y aplicando la política óptima $\pi^*$.

In [ ]:
# Configuración de la simulación
numero_escenarios = 10000  # Aumentamos para mayor precisión
semanas_a_estudiar = 8
estado_inicial = "Media"

# Mapeo de estados a índices
estado_a_indice = {estado: idx for idx, estado in enumerate(s_x)}

# Contador de escenarios con al menos una presión Alta
contador_alta = 0

# Semilla para reproducibilidad
np.random.seed(0)

# Simulación de Monte Carlo
for _ in range(numero_escenarios):
    estado_actual = estado_inicial
    presion_alta = False
    
    for semana in range(semanas_a_estudiar):
        # Obtener la acción según la política óptima
        accion = optimal_policy[estado_actual]
        
        # Obtener índice del estado actual
        estado_idx = estado_a_indice[estado_actual]
        
        # Transición al siguiente estado
        dist_prob = matrices[accion][estado_idx]
        estado_actual = np.random.choice(s_x, p=dist_prob)
        
        # Verificar si alcanzó presión Alta
        if estado_actual == "Alta":
            presion_alta = True
            
    # Incrementar contador si hubo al menos una presión Alta
    if presion_alta:
        contador_alta += 1

### 1. ¿Cual es el ingreso acumulado espeardo (en miles de dólares) durante las 25 semanas?

In [38]:
# Cálculo del ingreso acumulado esperado
ingreso_acumulado_esperado = np.mean(np.sum(ingresos, axis=0))

print(f"El ingreso acumulado esperado en {numero_semanas} semanas es: ${ingreso_acumulado_esperado:,.2f} (en miles de dólares)")

El ingreso acumulado esperado en 25 semanas es: $1,844.18 (en miles de dólares)


### 2. ¿Cuál es la probabilidad estimada de que Walter sienta una presión Alta al menos una vez en las primeras 8 semanas?

In [39]:
# Calcular probabilidad
probabilidad_alta = (contador_alta / numero_escenarios) * 100

print(f"Probabilidad de presión Alta en las primeras {semanas_a_estudiar} semanas: {probabilidad_alta:.2f}%")

Probabilidad de presión Alta en las primeras 8 semanas: 88.14%
